<a href="https://colab.research.google.com/github/simonamador/EEG-Channel-Selection-Algorithm/blob/main/MNEDataExtractionSubjectArray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

from google.colab import drive
drive.mount('/content/drive')
import mne
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Vector channels incluye los nombres de los canales que se deben
# extraer del archivo edf. Tambien se definen diccionarios de los indices con 
# nombres usados en libreria mne para poder añadir un montaje digital 
# (marcar las posiciones de los electrodos).

channels = ['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 
            'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.',
            'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 
            'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..',
            'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 
            'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..',
            'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..','Po7.',
            'Po3.','Poz.','Po4.','Po8.','O1..','Oz..','O2..','Iz..']

ch_rename64 = {'Fc5.':'FC5', 'Fc3.':'FC3', 'Fc1.':'FC1', 'Fcz.':'FCz', 
                  'Fc2.':'FC2', 'Fc4.':'FC4', 'Fc6.':'FC6', 'C5..':'C5', 
                  'C3..':'C3', 'C1..':'C1', 'Cz..':'Cz', 'C2..':'C2', 
                  'C4..':'C4', 'C6..':'C6', 'Cp5.':'CP5', 'Cp3.':'CP3', 
                  'Cp1.':'CP1', 'Cpz.':'CPz', 'Cp2.':'CP2', 'Cp4.':'CP4', 
                  'Cp6.':'CP6', 'Fp1.':'Fp1', 'Fpz.':'Fpz', 'Fp2.':'Fp2', 
                  'Af7.':'AF7', 'Af3.':'AF3', 'Afz.':'AFz', 'Af4.':'AF4', 
                  'Af8.':'AF8', 'F7..':'F7', 'F5..':'F5', 'F3..':'F3', 
                  'F1..':'F1', 'Fz..':'Fz', 'F2..':'F2', 'F4..':'F4', 
                  'F6..':'F6', 'F8..':'F8', 'Ft7.':'FT7', 'Ft8.':'FT8', 
                  'T7..':'T7', 'T8..':'T8', 'T9..':'T9', 'T10.':'T10', 
                  'Tp7.':'TP7', 'Tp8.':'TP8', 'P7..':'P7', 'P5..':'P5',
                  'P3..':'P3', 'P1..':'P1', 'Pz..':'Pz', 'P2..':'P2', 
                  'P4..':'P4', 'P6..':'P6', 'P8..':'P8','Po7.':'PO7',
                  'Po3.':'PO3','Poz.':'POz','Po4.':'PO4','Po8.':'PO8',
                  'O1..':'O1','Oz..':'Oz','O2..':'O2','Iz..':'Iz'}

In [ ]:
def importdata(suj,tar,f=0,channels=None):
# EXTRACCIÓN
#Pequeño hilo de condicionales para identificar el nombre a utilizar acorde a la iteración
  if suj<10 and tar<10:
    name = '/content/drive/MyDrive/Colab Notebooks/BaseDatos/S00'+str(suj)+'R0'+str(tar)+'.edf'
  elif suj<10 and tar>=10:
    name = '/content/drive/MyDrive/Colab Notebooks/BaseDatos/S00'+str(suj)+'R'+str(tar)+'.edf'
  elif 10<=suj<100 and tar<10:
    name = '/content/drive/MyDrive/Colab Notebooks/BaseDatos/S0'+str(suj)+'R0'+str(tar)+'.edf'
  elif 10<=suj<100 and tar>=10:
    name = '/content/drive/MyDrive/Colab Notebooks/BaseDatos/S0'+str(suj)+'R'+str(tar)+'.edf'
  elif 100<=suj and tar<10:
    name = '/content/drive/MyDrive/Colab Notebooks/BaseDatos/S'+str(suj)+'R0'+str(tar)+'.edf'
  else:
    name = '/content/drive/MyDrive/Colab Notebooks/BaseDatos/S'+str(suj)+'R'+str(tar)+'.edf'
  #Se extraen los datos usando read_edf a objeto raw, se añade artificialmente
  #un montaje estándar internacional 10-20
  raw = mne.io.read_raw_edf(name, preload = True, include=channels)

#FILTROS
  raw = raw.filter(8,33) #Filtro frecuencial, ondas beta y mu
  
  #ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
  #ica.fit(raw.filter(8,33))
  #ica.apply(raw)

#SEGMENTACIÓN
  #Se generan las matrices T0, T1 y T2
  events = mne.events_from_annotations(raw) #Obtiene eventos de anotaciones
  epochs = mne.Epochs(raw,events[0],event_id=events[1]) #Segmentación
  epochs.apply_baseline((None,0)) #Se usa 0 como base de eventos
  T0 = epochs['T0'].get_data() #Matriz de eventos de reposo
  T1 = epochs['T1'].get_data() #Matriz de eventos de tarea tipo 1
  T2 = epochs['T2'].get_data() #Matriz de eventos de tarea tipo 2
  return T0, T1, T2

In [ ]:
suj = 109 #Cantidad de sujetos 
tar = 14 #Cantidad de tareas

#Se generan matrices de tareas y etiquetas

fdata = []
ftask= []
#Se itera por sujeto y por tarea. Se adjuntan los T1,T2,T3 y sus respectivas
#etiquetas para cada tarea y cada sujeto en tres grandes matrices de estas.

#ETIQUETAS:
# 0 = reposo, 1 = mano izquierda, 2 = mano derecha, 
# 3 = mano izquierda imaginado, 4 = mano derecha imaginado, 
# 5 = ambas manos, 6 = ambos pies,
# 7 = ambas manos imaginado, 8 = ambos pies imaginado. 

for s in range(suj):
#Se generan matrices de tareas y etiquetas por sujeto
  data = np.empty(shape=(1,len(channels),113)) #tareas
  task = np.empty(shape=(1)) #etiquetas
  for t in range(tar):
#Condicional para excluir tres sujetos con eventos cortos
    if s in [87,91,99]:
      continue
    else:
      if t+1 in [1,2]: #Se descarta el REPOSO, ya que se usa el reposo de los segmentos
        continue
      else:
        T0, T1, T2 = importdata(s+1,t+1,f=1,channels=channels) #Se extraen los datos
        if t+1 in [3,7,11]: #Abre/cierra puño derecho/izquierdo, 
#ETIQUETAS
          task = np.append(task,np.repeat(0,T0.shape[0]))
          task = np.append(task,np.repeat(1,T1.shape[0]))
          task = np.append(task,np.repeat(2,T2.shape[0]))
        elif t+1 in [4,8,12]: #Abre/cierra puño derecho/izquierdo imaginado
#ETIQUETAS 
          task = np.append(task,np.repeat(0,T0.shape[0]))
          task = np.append(task,np.repeat(3,T1.shape[0]))
          task = np.append(task,np.repeat(4,T2.shape[0]))
        elif t+1 in [5,9,13]: #Abre/cierra puños/pies
#ETIQUETAS
          task = np.append(task,np.repeat(0,T0.shape[0]))
          task = np.append(task,np.repeat(5,T1.shape[0]))
          task = np.append(task,np.repeat(6,T2.shape[0]))
        elif t+1 in [6,10,14]: #Abre/cierra puños/pies imaginado
#ETIQUETAS
          task = np.append(task,np.repeat(0,T0.shape[0]))
          task = np.append(task,np.repeat(7,T1.shape[0]))
          task = np.append(task,np.repeat(8,T2.shape[0]))
#TAREAS
        data = np.append(data,T0,axis=0)
        data = np.append(data,T1,axis=0)
        data = np.append(data,T2,axis=0)
#Se elimina la primera fila de las matrices, ya que esta vacía
  data = np.delete(data, 0, 0) #Tareas
  task = np.delete(task, 0, 0) #Etiquetas
  fdata.append(data)
  ftask.append(task)

Extracting EDF parameters from /content/drive/MyDrive/Colab Notebooks/BaseDatos/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 33.00 Hz
- Upper transition bandwidth: 8.25 Hz (-6 dB cutoff frequency: 37.12 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items ac

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 33.00 Hz
- Upper transition bandwidth: 8.25 Hz (-6 dB cutoff frequency: 37.12 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 15 events and 113 original ti

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 33.00 Hz
- Upper transition bandwidth: 8.25 Hz (-6 dB cutoff frequency: 37.12 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 15 events and 113 original ti

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 33.00 Hz
- Upper transition bandwidth: 8.25 Hz (-6 dB cutoff frequency: 37.12 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 15 events and 113 original ti

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


0 bad epochs dropped
Extracting EDF parameters from /content/drive/MyDrive/Colab Notebooks/BaseDatos/S109R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 33.00 Hz
- Upper transition bandwidth: 8.25 Hz (-6 dB cutoff frequency: 37.12 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


In [ ]:
#Se guardan las cuatro matrices como archivos comprimidos de numpy
np.savez_compressed('SubjectsData', X = np.array(fdata, dtype=object), y = np.array(ftask, dtype=object), allow_pickle=True)